**Sommario**

- [`flask_login_4` - Database con SQLite](#flask_login_4---database-con-sqlite)
  - [Struttura del progetto aggiornata](#struttura-del-progetto-aggiornata)
  - [Configurazione del database in `settings.py`](#configurazione-del-database-in-settingspy)
  - [Gestione del database in `db.py`](#gestione-del-database-in-dbpy)
    - [1. **Importazioni**](#1-importazioni)
    - [2. **Mock data per i Film**](#2-mock-data-per-i-film)
    - [3. **Recupero della password dell'Utente**](#3-recupero-della-password-dellutente)
      - [Descrizione generale della funzione `get_user_password()`](#descrizione-generale-della-funzione-get_user_password)
      - [Blocco `try ... except`](#blocco-try--except)
      - [Connessione al database](#connessione-al-database)
      - [Creazione di un cursore](#creazione-di-un-cursore)
      - [Preparazione della query SQL per la ricerca dell'Utente](#preparazione-della-query-sql-per-la-ricerca-dellutente)
      - [Esecuzione della query](#esecuzione-della-query)
      - [Recupero del risultato](#recupero-del-risultato)
      - [Chiusura esplicita del cursore](#chiusura-esplicita-del-cursore)
      - [Controllo del risultato e restituzione della password](#controllo-del-risultato-e-restituzione-della-password)
    - [4. **Verifica dell'esistenza di una tabella**](#4-verifica-dellesistenza-di-una-tabella)
      - [Descrizione generale della funzione `table_exists()`](#descrizione-generale-della-funzione-table_exists)
      - [Parametri della funzione](#parametri-della-funzione)
      - [Esecuzione della query SQL](#esecuzione-della-query-sql)
      - [Recupero del risultato](#recupero-del-risultato)
    - [5. **Creazione della tabella Utente**](#5-creazione-della-tabella-utente)
    - [6. **Inizializzazione della tabella Utente**](#6-inizializzazione-della-tabella-utente)
      - [Descrizione generale della funzione `_init_user_table()`](#descrizione-generale-della-funzione-_init_user_table)
      - [Verifica dell'esistenza del file CSV](#verifica-dellesistenza-del-file-csv)
      - [Blocco `try ... except`](#blocco-try--except)
      - [Connessione al database](#connessione-al-database)
      - [Creazione di un cursore](#creazione-di-un-cursore)
      - [Verifica dell'esistenza della tabella Utente](#verifica-dellesistenza-della-tabella-utente)
      - [Creazione della tabella Utente](#creazione-della-tabella-utente)
      - [Apertura del file CSV](#apertura-del-file-csv)
      - [Lettura del file CSV](#lettura-del-file-csv)
      - [Preparazione della query SQL per l'inserimento dei dati](#preparazione-della-query-sql-per-linserimento-dei-dati)
      - [Popolamento della tabella Utente](#popolamento-della-tabella-utente)
  - [Usare il DB in `app.py`](#usare-il-db-in-apppy)
    - [7. **Importazioni**](#7-importazioni)
    - [8. **Inizializzazione del database**](#8-inizializzazione-del-database)
    - [9. **Route per il login e controllo credenziali**](#9-route-per-il-login-e-controllo-credenziali)

# `flask_login_4` - Database con SQLite

Esercitazione INF_PR_PY_WB_E07.

In questo esempio implementiamo la gestione degli Utenti con un database SQLite. In particolare vedremo come:

- predisporre la nostra app Flask all'uso di un database SQLite;

- impostare i parametri di configurazione del database nel file `settings.py`;

- scrivere funzioni nel file `db.py` per gestire le connessioni al DB e l'inizializzazione delle tabelle;

- utilizzare queste funzioni nel file `app.py` per gestire le rotte e l'autenticazione degli utenti.

SQLite è il più semplice motore di database che abbiamo a disposizione e viene installato automaticamente quando installiamo Python. `sqlite3` è un modulo della Libreria standard.

Per usare SQLite direttamente con il modulo `sqlite3` dobbiamo scrivere noi tutte le query SQL per gestire le nostre tabelle e i nostri dati.

## Struttura del progetto aggiornata

In questa quinta fase modifichiamo ancora una volta la struttura del progetto in questo modo:

```text
flask_login_4/
│
├── app.py                     << ADATTATO AL DB
├── db.py                      << NUOVA IMPLEMENTAZIONE
├── settings.py                << NUOVO
├── database/                  << NUOVO
│   ├── db.sqlite3             << NUOVO
│   └── users.csv              << NUOVO
├── static/
│   ├── script.js
│   ├── style.css
│   └── imgs/
│       ├── akira.jpg
│       ├── blade-runner.jpg
│       ├── gits.jpg
│       ├── hackers.jpg
│       ├── nirvana.jpg
│       └── shortcut-icon.png  
└── templates/
    ├── base.html
    ├── films.html
    ├── home.html
    ├── login.html
    └── includes/
        ├── flash.html
        ├── footer.html
        ├── head.html
        └── navbar.html
```


## Configurazione del database in `settings.py`

Il file `settings.py` contiene configurazioni cruciali come il percorso al database e al file CSV per l'inizializzazione della tabella Utenti.

Il file `settings.py` appare così:

```python
import os

# Ottiene il percorso assoluto alla cartella che contiene questo file
BASE_DIR = os.path.abspath(os.path.dirname(__file__))

# Percorso assoluto al file del database SQLite
DATABASE = os.path.join(BASE_DIR, 'database', 'db.sqlite3')

# Percorso assoluto al file CSV per la tabelle utenti
USER_TABLE_CSV = os.path.join(BASE_DIR, 'database', 'users.csv')

# Nome della tabella
USER_TABLE_NAME = 'user'
```

In questo script:

- `os.path` è un modulo per la manipolazione dei percorsi di file e cartelle.

- `os.path.dirname(__file__)` restituisce il percorso assoluto alla cartella contenente il file `settings.py`.

- `os.path.abspath()` assicura di ottenere un percorso assoluto.

- `os.path.join()` crea un percorso combinando più percorsi parziali; in questo caso crea i percorsi per il file CSV con i dati (`users.csv`) da importare e per il file del database vero e proprio (`db.sqlite3`).

## Gestione del database in `db.py`

Il file `db.py` contiene le funzioni per gestire il database SQLite. Include funzioni per la creazione di tabelle, l'inizializzazione del database e il recupero delle password degli utenti.

Vediamo il file `db.py`.

### 1. **Importazioni**

Importiamo i moduli necessari e configuriamo il database:

```python
import sys
import os
import csv
import sqlite3
from settings import DATABASE, USER_TABLE_NAME, USER_TABLE_CSV
...
```

### 2. **Mock data per i Film**

Per ora, utilizziamo ancora una lista di dizionari per simulare il database di film:

```python
FILMS = [
    {'title': 'Akira', 'image': 'akira.jpg'},
    {'title': 'Ghost in the Shell', 'image': 'gits.jpg'},
    {'title': 'Blade Runner', 'image': 'blade-runner.jpg'},
    {'title': 'Hackers', 'image': 'hackers.jpg'},
    {'title': 'Nirvana', 'image': 'nirvana.jpg'},
]
```

### 3. **Recupero della password dell'Utente**

Funzione per recuperare la password di un utente dal database:

```python
def get_user_password(username):
    """
    Cerca un utente nel database sulla base del suo username e restituisce
    la sua password se l'utente è presente, altrimenti `None`.
    
    :param username: Lo username dell'utente da cercare.
    :return: La password dell'utente se trovato, altrimenti None.
    """
    try:
        with sqlite3.connect(DATABASE) as conn:
            cursor = conn.cursor()
            query = f"SELECT PASSWORD FROM {USER_TABLE_NAME} WHERE LOGIN = ?"
            cursor.execute(query, (username,))
            result = cursor.fetchone()
            cursor.close()
            if result:
                return result[0]
            else:
                return None
    except sqlite3.Error as err:
        print(f"Si è verificato un errore durante l'accesso al database: {err}")
        return None
```


#### Descrizione generale della funzione `get_user_password()`

Come riporta la docstring all'inizio della funzione, la quale spiega cosa fa la funzione, quali sono i parametri di input e cosa restituisce, la funzione `get_user_password()` fa quanto segue:

```python
"""
Cerca un utente nel database sulla base del suo username e restituisce
la sua password se l'utente è presente, altrimenti `None`.

:param username: Lo username dell'utente da cercare.
:return: La password dell'utente se trovato, altrimenti None.
"""
```
In altre parole:
- Accetta un parametro, `username`, che è il nome dell'Utente.
- Restituisce la password dell'Utente se questo esiste altrimenti `None`.

#### Blocco `try ... except`

Il blocco `try ... except` è utilizzato per gestire eventuali errori che possono verificarsi durante l'accesso al database. 

```python
try:
    ...  # Esegue la query di ricerca
except sqlite3.Error as err:
    print(f"Si è verificato un errore durante l'accesso al database: {err}")
    return None
```

Se si verifica un errore durante l'accesso al database, il blocco `except` cattura l'eccezione e stampa un messaggio di errore.

La funzione restituisce `None` in caso di errore.

#### Connessione al database

Si utilizza il metodo `sqlite3.connect()` per connettersi al database specificato nella variabile `DATABASE`. La connessione viene gestita utilizzando un contesto `with`, il che assicura che la connessione venga chiusa automaticamente quando il blocco `with` termina.

```python
with sqlite3.connect(DATABASE) as conn:
    ...
```

#### Creazione di un cursore

Viene creato un cursore per eseguire le operazioni SQL.

> **NOTA**: Un _**cursore**_ è un oggetto che permette di interagire con il database ed eseguire query SQL.

```python
cursor = conn.cursor()
```

#### Preparazione della query SQL per la ricerca dell'Utente

Viene preparata una query SQL per selezionare la password dell'utente basata sullo username fornito.

> **NOTA**: La query utilizza un placeholder `?` per prevenire attacchi di SQL injection.

```python
query = f"SELECT PASSWORD FROM {USER_TABLE_NAME} WHERE LOGIN = ?"
```

#### Esecuzione della query

La query viene eseguita utilizzando il metodo `query.execute` del cursore.

> **NOTA**: Viene passata un tupla contenente lo username, il quale sarà usato come parametro per il placeholder `?`.

```python
cursor.execute(query, (username,))
#                     ^^^tupla^^^
```

#### Recupero del risultato

Viene utilizzato il metodo `cursor.fetchone()` per recuperare la prima riga del risultato della query. Se non ci sono risultati, `.fetchone()` restituisce `None`.

```python
result = cursor.fetchone()
```

#### Chiusura esplicita del cursore

Il cursore viene chiuso esplicitamente per liberare le risorse. Anche se il cursore verrà chiuso automaticamente alla fine del blocco `with`, chiuderlo esplicitamente è una buona pratica.

```python
cursor.close()
```

#### Controllo del risultato e restituzione della password

Se `result` contiene un valore (ovvero, l'utente è stato trovato), la funzione restituisce la password (il primo elemento di `result`). Altrimenti, restituisce `None`.

```python
if result:
    return result[0]
else:
    return None
```

### 4. **Verifica dell'esistenza di una tabella**

Dato che sapere se una tabella esiste all'interno del database è una cosa utile che potremmo dover fare più volte, è una buona idea creare una funziona apposita che si occupi di questo.

Ecco come potrebbe essere una funzione che verifica se una tabella esiste nel database:

```python
def table_exists(cursor, table_name):
    """
    Controlla se una tabella esiste nel database.
    :param cursor: Il cursore per eseguire le query.
    :param table_name: Il nome della tabella da cercare.
    :return: True se la tabella esiste, False altrimenti.
    """
    cursor.execute('''
        SELECT name FROM sqlite_master
        WHERE type='table' AND name=?;
    ''', (table_name,))
    return cursor.fetchone() is not None
```

#### Descrizione generale della funzione `table_exists()`

Come leggiamo nella docstring, la funzione `table_exists()` verifica se una specifica tabella esiste nel database SQLite.

In altre parole:
- Accetta due parametri, `cursor` e `table_name`.
- Restituisce `True` se la tabella esiste altrimenti `False`.

#### Parametri della funzione

La funzione prende due parametri:
- `cursor`: un cursore del database per eseguire le query.
- `table_name`: il nome della tabella da verificare.

#### Esecuzione della query SQL

Utilizziamo il metodo `.execute()` del cursore per eseguire una query SQL che verifica l'esistenza della tabella nel database SQLite. La query verifica la presenza di una tabella con il nome specificato nella tabella `sqlite_master`, che contiene le informazioni su tutte le tabelle del database.

```python
cursor.execute('''
    SELECT name FROM sqlite_master
    WHERE type='table' AND name=?;
''', (table_name,))
```

- **`SELECT name FROM sqlite_master WHERE type='table' AND name=?`**: Questa query seleziona il `name` della tabella dalla tabella di sistema `sqlite_master` dove il tipo è `'table'` e il nome della tabella corrisponde al parametro passato.
- **`(table_name,)`**: Una tupla contenente il nome della tabella viene passata come parametro in modo che questo valore venga inserito al posto del placeholder `?` (questo per prevenire attacchi di SQL injection.).

#### Recupero del risultato

Utilizziamo di nuovo il metodo `.fetchone()` del cursore per recuperare una riga dal risultato della query.

```python
return cursor.fetchone() is not None
```

L'espressione `cursor.fetchone() is not None` viene valutata `True` se `cursor.fetchone()` restituisce un record, altrimenti viene valutato `False`.

### 5. **Creazione della tabella Utente**

Anche in questo caso, per maggiore modularità, scriviamo una funzione per creare la tabella Utente nel database:

```python
def _create_user_table(cursor):
    """
    Crea la tabella utente nel database.
    :param cursor: Il cursore per eseguire le query.
    """
    cursor.execute(f'''
        CREATE TABLE {USER_TABLE_NAME}
        (ID INTEGER PRIMARY KEY AUTOINCREMENT,
        LOGIN TEXT NOT NULL UNIQUE,
        PASSWORD TEXT NOT NULL,
        NOME TEXT,
        COGNOME TEXT,
        INDIRIZZO TEXT,
        CITTA TEXT,
        TEL1 TEXT,
        TEL2 TEXT,
        EMAIL TEXT,
        DATANASCITA DATE,
        DATAREG DATE);
    ''')

```

La funzione accetta un solo parametro `cursor` e non restituisce nulla. Esegue semplicemente la query di creazione.

In questo caso non usiamo il segnaposto `?` perché il nome della tabella da creare è una costante e arriva dalla configurazione interna della nostra app.

I vari campi vengono creati seguendo le specifiche indicate sul testo dell'esercitazione INF_PR_PY_WB_E07.

### 6. **Inizializzazione della tabella Utente**

Infine creiamo una funzione per inizializzare la tabella Utente nel database.

L'inizializzazione consiste di due fasi:

1. Creazione della tabella.

2. Popolamento con i primi dati provenienti da un file CSV.

Ecco come potrebbe essere una funzione di inizializzazione.

```python
def _init_user_table():
    """
    Inizializza la tabella degli utenti nel database e la popola con i dati
    contenuti nel file CSV `users.csv`.
    """
    if os.path.exists(USER_TABLE_CSV):
        try:
            with sqlite3.connect(DATABASE) as conn:
                cursor = conn.cursor()
                if not table_exists(cursor, USER_TABLE_NAME):
                    _create_user_table(cursor)
                    with open(USER_TABLE_CSV, 'r') as file:
                        csv_dict_reader = csv.DictReader(file)
                        user_query = f'''
                            INSERT INTO {USER_TABLE_NAME}
                            (LOGIN, PASSWORD, NOME, COGNOME,
                            INDIRIZZO, CITTA, TEL1, TEL2,
                            EMAIL, DATANASCITA, DATAREG)
                            VALUES (:LOGIN, :PASSWORD, :NOME, :COGNOME,
                            :INDIRIZZO, :CITTA, :TEL1, :TEL2,
                            :EMAIL, :DATANASCITA, :DATAREG);
                        '''
                        cursor.executemany(user_query, csv_dict_reader)
                    print(
                        'La tabella utente è stata inizializzata e popolata '
                        'con successo.'
                    )
                else:
                    print(
                        'La tabella utente esiste già. Non è necessario '
                        'inizializzarla.'
                    )
        except sqlite3.Error as err:
            print(f'Si è verificato un errore del database: {err}')
            sys.exit(1)
        except Exception as err:
            print(f'Si è verificato un errore generico: {err}')
            sys.exit(1)
    else:
        print(
            f'Il file "{USER_TABLE_CSV}" non esiste. '
            'Verifica il percorso e riprova.'
        )
        sys.exit(1)
```

#### Descrizione generale della funzione `_init_user_table()`

Come leggiamo nella docstring, la funzione `_init_user_table()` inizializza la tabella degli utenti nel database SQLite e la popola con i dati contenuti in un file CSV `users.csv`.

Inoltre, in caso di errore, questo viene intercettato e nel terminale vengono mostrate informazioni aggiuntive; dopodiché il programma viene terminato.

#### Verifica dell'esistenza del file CSV

La funzione inizia verificando se il file CSV esiste utilizzando `os.path.exists`.

```python
if os.path.exists(USER_TABLE_CSV):
    ...
else:
    print(f'Il file "{USER_TABLE_CSV}" non esiste. Verifica il percorso e riprova.')
    sys.exit(1)
```

Se il file non esiste, il programma termina con `sys.exit(1)` dopo aver mostrato un messaggio utile ai fini di debug.

#### Blocco `try ... except`

Il blocco `try ... except` è utilizzato per gestire eventuali errori durante l'accesso al database e la lettura del file CSV.

```python
try:
    ...
except sqlite3.Error as err:
    print(f'Si è verificato un errore del database: {err}')
    sys.exit(1)
except Exception as err:
    print(f'Si è verificato un errore generico: {err}')
    sys.exit(1)

```

Vengono intercettati due tipi di errori:

- `sqlite3.Error`, in caso si verifichi un errore dal database.
- `Exception`, in caso si verifichi un qualunque altro tipo di errore.

In entrambi i casi il programma viene terminato con `sys.exit(1)` dopo aver mostrato un messaggio con ulteriori informazioni nel terminale.

#### Connessione al database

Si utilizza il metodo `sqlite3.connect()` per connettersi al database specificato nella costante `DATABASE`. La connessione viene gestita utilizzando un contesto `with`, il che assicura che la connessione venga chiusa automaticamente quando il blocco `with` termina.

```python
with sqlite3.connect(DATABASE) as conn:
    ...
```

#### Creazione di un cursore

Viene creato un cursore con il quale poter eseguire le query SQL.

```python
cursor = conn.cursor()
```

#### Verifica dell'esistenza della tabella Utente

Si utilizza la funzione `table_exists()` che abbiamo definito precedentemente per verificare se la tabella utente esiste già nel database in quanto non vogliamo eseguire la query di creazione se la tabella esiste già.

```python
if not table_exists(cursor, USER_TABLE_NAME):
    ... # Inizializzazione tabella
    print('La tabella utente è stata inizializzata e popolata con successo.')
else:
    print('La tabella utente esiste già. Non è necessario inizializzarla.')
```

Al termine di ciascuna condizione, viene visualizzato un messaggio che indica cosa è avvenuto.

#### Creazione della tabella Utente

Se la tabella utente non esiste, viene creata utilizzando la funzione `_create_user_table()` che abbiamo definito precedentemente.

```python
_create_user_table(cursor)
```

#### Apertura del file CSV

L'esercizio fornisce una tabella in formato CSV contenente i dati degli Utenti. Il file si trova in `database/users.csv`.

Il file CSV viene aperto in modalità lettura utilizzando un contesto `with`, il che assicura la chiusura del file al termine del blocco di codice.

```python
with open(USER_TABLE_CSV, 'r') as file:
    ...
```

#### Lettura del file CSV

Si utilizza la classe `csv.DictReader` per leggere il file CSV contenente i dati degli Utenti.

`DictReader` ci restituisce direttamente un comodo dizionario le cui chiavi chiavi corrispondono ai nomi dei campi/colonne della tabella.

```python
csv_dict_reader = csv.DictReader(file)
```

#### Preparazione della query SQL per l'inserimento dei dati

Viene preparata una query SQL per l'inserimento dei dati nella tabella utente.

La query utilizza parametri nominati per prevenire attacchi di SQL injection.

```python
user_query = f'''
    INSERT INTO {USER_TABLE_NAME}
    (LOGIN, PASSWORD, NOME, COGNOME,
    INDIRIZZO, CITTA, TEL1, TEL2,
    EMAIL, DATANASCITA, DATAREG)
    VALUES (:LOGIN, :PASSWORD, :NOME, :COGNOME,
    :INDIRIZZO, :CITTA, :TEL1, :TEL2,
    :EMAIL, :DATANASCITA, :DATAREG);
'''
```

> **NOTA**: Dovendo usare un dizionario come struttura dati che contiene i valori (`VALUES`), la sintassi dei placeholder (segnaposto) è `:NOME_CAMPO` anziché `?`, che viene usato se la struttura dati è una sequenza come una tupla o una lista.

#### Popolamento della tabella Utente

La tabella viene popolata eseguendo la query per ogni riga del file CSV utilizzando il metodo `cursor.executemany()`.

```python
cursor.executemany(user_query, csv_dict_reader)
```

## Usare il DB in `app.py`

Questo file contiene la logica dell'applicazione Flask, compresa la gestione delle rotte, l'autenticazione degli utenti e la visualizzazione dei film.

### 7. **Importazioni**

Importiamo gli oggetti necessari dal modulo `db` (che è il file `db.py`):

```python
from flask import Flask, ...
from db import _init_user_table, get_user_password, FILMS
...
```

### 8. **Inizializzazione del database**

Inizializziamo la tabella Utente subito prima dell'avvio dell'applicazione:

```python
if __name__ == '__main__':

    # Utilizziamo la funzione creata in db.py per inizializzare la tabella utenti
    _init_user_table()

    app.run(debug=True)
```

### 9. **Route per il login e controllo credenziali**

Definiamo la route per il login, che utilizza la funzione `get_user_password()` per verificare le credenziali dell'Utente:

```python
@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        rx_username = request.form.get('tx_user')
        rx_password = request.form.get('tx_password')

        # Utilizziamo la funzione creata in db.py per ottenere la password
        db_password = get_user_password(rx_username)

        if rx_password == db_password:
            ...
        else:
            ...
    return render_template('login.html')
```